# 🎥 Real-ESRGAN Video Upscaler
This notebook allows you to upload a low-quality video, enhance it using Real-ESRGAN, and download the final version with audio preserved.

In [1]:
from IPython.display import Javascript
from urllib.parse import unquote
import time

video_url = ""

def get_param(name):
    js_code = f"""
    async function getParam() {{
      const url = new URL(window.location.href);
      const param = url.searchParams.get("{name}");
      if (param) {{
        google.colab.kernel.invokeFunction('notebook.get_param', [param], {{}});
      }}
    }}
    getParam();
    """
    display(Javascript(js_code))

def set_param(value):
    global video_url
    video_url = unquote(value)
    print(f"✅ Video URL set from query param: {video_url}")

import google.colab.output
google.colab.output.register_callback('notebook.get_param', set_param)

get_param("video_url")
time.sleep(2)  # Wait for JS to inject value


<IPython.core.display.Javascript object>

In [ ]:
if video_url:
    # Proceed to download it
    print(f"Downloading video from: {video_url}")
    import requests
    r = requests.get(video_url, stream=True)
    with open("input.mp4", "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    print("✅ Downloaded video as input.mp4")
else:
    print("⚠️ No video URL. Please upload manually below.")


In [2]:
#@title 🛠️ Setup
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
!pip -q install basicsr facexlib gfpgan
!pip -q install -r requirements.txt
!python setup.py develop
!pip install torch==2.1.0 torchvision==0.16.0 --index-url https://download.pytorch.org/whl/cu118
!pip install "numpy<2.0"


Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 759, done.
remote: Total 759 (delta 0), reused 0 (delta 0), pack-reused 759 (from 1)
Receiving objects: 100% (759/759), 5.39 MiB | 36.28 MiB/s, done.
Resolving deltas: 100% (408/408), done.
/content/Real-ESRGAN
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 17.3 MB/s eta 0:00:00
/usr/loc

In [ ]:
#@title 🖼️ Extract Frames
import cv2, os
frames_dir = 'input_frames'
os.makedirs(frames_dir, exist_ok=True)
cap = cv2.VideoCapture(video_url)
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_path = os.path.join(frames_dir, f"{frame_count:05d}.png")
    cv2.imwrite(frame_path, frame)
    frame_count += 1
cap.release()
print(f"🖼️ Extracted {frame_count} frames to '{frames_dir}'")


> ✅ Now manually run Real-ESRGAN on extracted frames OR set output folder below.

In [ ]:
#@title 🚀 Enhance Frames using Real-ESRGAN
!python inference_realesrgan.py -n RealESRGAN_x4plus -i input_frames -o results


In [ ]:
#@title 🧪 Combine Upscaled Frames
import glob
output_folder = 'results'
upscaled_frames = sorted(glob.glob(os.path.join(output_folder, '*.png')))
if not upscaled_frames:
    raise ValueError("❌ No upscaled frames found!")
first_frame = cv2.imread(upscaled_frames[0])
height, width, _ = first_frame.shape
upscaled_video_path = 'upscaled_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = 24
out = cv2.VideoWriter(upscaled_video_path, fourcc, fps, (width, height))
for frame_path in upscaled_frames:
    frame = cv2.imread(frame_path)
    out.write(frame)
out.release()
print(f"🎉 Video saved to: {upscaled_video_path}")


In [ ]:
#@title 🔊 Merge with Original Audio
final_output = "final_upscaled_with_audio.mp4"
!ffmpeg -y -i "{upscaled_video_path}" -i "{video_path}" -c copy -map 0:v:0 -map 1:a:0 -shortest "{final_output}"


In [ ]:
# ✅ Final UI "Popup" Style Section
from IPython.display import display, HTML

html_popup = f"""
<div style="border: 2px solid #4CAF50; border-radius: 10px; padding: 20px; background-color: #f9fff9; box-shadow: 0 4px 12px rgba(0,0,0,0.1); max-width: 700px; margin: 20px auto; font-family: sans-serif;">
  <h2 style="color: #2e7d32;">✅ Enhancement Complete!</h2>
  <p>Your enhanced video is ready. Preview it below or click to download:</p>
  <video width="100%" controls style="border-radius: 8px;">
    <source src="final_output.mp4" type="video/mp4">
    Your browser does not support the video tag.
  </video>
  <br><br>
  <a href="final_output.mp4" download style="background-color: #4CAF50; color: white; padding: 10px 20px; text-decoration: none; border-radius: 5px; font-size: 16px;">
    📥 Download Enhanced Video
  </a>
</div>
"""

display(HTML(html_popup))


In [ ]:
#@title 📥 Download Final Video
from google.colab import files
files.download("final_upscaled_with_audio.mp4")
